IMPORTANDO SPARK

In [2]:
%pip install pyspark

from pyspark.sql import SparkSession

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession

# Create or get a SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/22 09:38:03 WARN Utils: Your hostname, 8X06064, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/22 09:38:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/22 09:38:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import SparkSession

# Garante que não há sessão antiga
try:
    spark.stop()
except:
    pass

spark = SparkSession.builder \
    .appName("SparkComJava11") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("✅ Spark ativo — versão:", spark.version)


✅ Spark ativo — versão: 4.0.0


DEFININDO TABELAS

In [5]:
df_clientes = spark.read.csv(
    "/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/clientes.csv",
    header=True,
    inferSchema=True
)
df_clientes.createOrReplaceTempView("df_clientes")
df_clientes.show()

+---+--------------------+--------+--------------------+------+
| id|        nome_cliente|    tipo|              cidade|estado|
+---+--------------------+--------+--------------------+------+
|  1|               Jesus|Farmácia|    Azevedo de Minas|    TO|
|  2|       Caldeira S.A.|Hospital|     Santos de Minas|    PR|
|  3|               Pires|Farmácia|     Pinto do Amparo|    CE|
|  4|          Nascimento| Clínica|              da Luz|    AP|
|  5|        Moraes Ltda.|Farmácia|            Teixeira|    SP|
|  6|             Almeida|Hospital|               Alves|    PR|
|  7|             Ribeiro| Clínica|     da Luz Paulista|    PE|
|  8|        Oliveira S/A|Hospital|              Araújo|    SC|
|  9|              Fogaça|Farmácia|        Aragão Verde|    SC|
| 10|          da Luz S/A| Clínica|    da Rosa do Campo|    ES|
| 11|      Rodrigues - EI|Farmácia|      Lopes de Minas|    RN|
| 12|               Pires| Clínica|                Melo|    ES|
| 13|Monteiro Rodrigue...| Clínica|     

In [6]:
df_vendas =  spark.read.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/vendas.csv", header=True, inferSchema=True)
df_vendas.createOrReplaceTempView("df_vendas")
df_vendas.show()

+---+----------+----------+----------+----------------+----------+-----------+
| id|data_venda|produto_id|cliente_id|representante_id|quantidade|valor_total|
+---+----------+----------+----------+----------------+----------+-----------+
|  1|03-11-2024|         5|        46|              26|        28|    8575.85|
|  2|06-04-2025|        57|        90|              18|        39|    8980.72|
|  3|10-05-2025|        45|        59|              13|        12|     8287.2|
|  4|02-11-2023|        58|        35|               8|        35|    5201.28|
|  5|11-09-2025|        10|        40|              30|        19|      958.7|
|  6|08-06-2024|        35|        33|               6|        21|    6725.63|
|  7|23-11-2024|         3|        30|              14|         7|    1929.33|
|  8|13-04-2024|        54|        16|              39|         6|    1492.06|
|  9|05-04-2025|        24|        93|               2|        21|    3303.66|
| 10|01-06-2025|        38|        25|              

In [7]:
df_representantes = spark.read.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/representantes.csv", header=True, inferSchema=True)
df_representantes.createOrReplaceTempView("df_representantes")
df_representantes.show()

+---+--------------------+------------+-----------+
| id|  nome_representante|      regiao|meta_mensal|
+---+--------------------+------------+-----------+
|  1|       Bruno Costela|         Sul|   89780.06|
|  2|     Marcela Rezende|     Sudeste|  189988.91|
|  3|    Joaquim da Costa|         Sul|  182129.63|
|  4|      Daniel Barbosa|       Norte|  181890.54|
|  5|    Dra. Eloah Pires|         Sul|  105429.06|
|  6|      Eduardo Fogaça|     Sudeste|   73662.02|
|  7|     Dr. Ian Costela|    Nordeste|  175061.74|
|  8|       Rebeca Araújo|         Sul|  155530.99|
|  9|        Felipe Silva|       Norte|  141751.66|
| 10|Dr. Luiz Otávio R...|     Sudeste|  198084.96|
| 11|         Cauã Castro|       Norte|  148096.45|
| 12|      Brenda Cardoso|       Norte|   51173.47|
| 13|       Leandro Jesus|         Sul|  172565.62|
| 14|      Antônio Fogaça|       Norte|   94906.81|
| 15|        Yasmin Lopes|         Sul|  149508.31|
| 16|        Luiza Moraes|    Nordeste|   190839.5|
| 17|       

In [8]:
df_produtos = spark.read.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/produtos.csv", header=True, inferSchema=True)
df_produtos.createOrReplaceTempView("df_produtos")
df_produtos.show()

+---+----------------+-----------------+--------------+-----------+
| id|    nome_produto|        categoria|preco_unitario| fabricante|
+---+----------------+-----------------+--------------+-----------+
|  1| Dipirona Sódica|        Vitaminas|        110.02|     Pfizer|
|  2|     Paracetamol|       Analgésico|         48.01|        EMS|
|  3|      Ibuprofeno|      Antibiótico|        123.15|Neo Química|
|  4|     Amoxicilina|Anti-inflamatório|         59.56|       Aché|
|  5|    Azitromicina|          Vacinas|         85.72|      Bayer|
|  6|     Cetoprofeno|        Vitaminas|        208.98|       Aché|
|  7|       Omeprazol|        Vitaminas|        206.57|  Eurofarma|
|  8|     Pantoprazol|          Vacinas|        228.36|        EMS|
|  9|      Loratadina|   Dermocosmético|        155.22|     Pfizer|
| 10|      Prednisona|          Vacinas|         34.83|      Bayer|
| 11|      Simeticona|        Vitaminas|        155.06|Neo Química|
| 12|       Losartana|        Vitaminas|        

In [9]:
df_medicos = spark.read.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/medicos.csv", header=True, inferSchema=True)
df_medicos.createOrReplaceTempView("df_medicos")
df_medicos.show()

+---+--------------------+--------+-------------+
| id|         nome_medico|     crm|especialidade|
+---+--------------------+--------+-------------+
|  1|       Theo da Cunha|46822/AL| Dermatologia|
|  2|Dr. Davi Lucas Costa|17679/PB|Clínico Geral|
|  3|    Dra. Júlia Gomes|71824/ES|Clínico Geral|
|  4|    Fernando Azevedo|79393/MG|    Ortopedia|
|  5|       Eduarda Moura|56849/MG|  Cardiologia|
|  6|      Alexia Pereira|86544/BA| Dermatologia|
|  7|         Elisa Lopes|40508/AM| Dermatologia|
|  8|      Isaac Caldeira|30922/GO|  Cardiologia|
|  9|     Otávio Oliveira|23124/SC|    Ortopedia|
| 10|       Laís Carvalho|42487/SE|  Cardiologia|
| 11|        Natália Dias|96096/DF|  Cardiologia|
| 12|       Mirella Silva|41608/BA|    Pediatria|
| 13|    Benjamin Pereira|44934/BA|    Pediatria|
| 14|      Esther Cardoso|79951/SE|    Ortopedia|
| 15|    Sra. Helena Lima|17328/MA|    Pediatria|
| 16|        Miguel Ramos|38758/RN|Clínico Geral|
| 17|        Sofia Vieira|85457/MT|    Pediatria|


In [10]:
df_laboratorios = spark.read.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/laboratorios_parceiros.csv", header=True, inferSchema=True)
df_laboratorios.createOrReplaceTempView("df_laboratorios")
df_laboratorios.show()

+---+--------------------+--------------------+----------------+
| id|    nome_laboratorio|                pais|   area_parceria|
+---+--------------------+--------------------+----------------+
|  1|           Fernandes|            Dhekelia|    Distribuição|
|  2|        Moreira S.A.|Ilhas Virgens Bri...|    Distribuição|
|  3|Viana Silveira Ltda.|               Benim|    Distribuição|
|  4|             da Mota|    Papua-Nova Guiné|    Distribuição|
|  5|             Costela|     Wallis e Futuna| Desenvolvimento|
|  6|        Farias Ltda.|            Dhekelia|Pesquisa Clínica|
|  7|      Ramos e Filhos|              Brunei| Desenvolvimento|
|  8|            Ferreira|               Aruba|Pesquisa Clínica|
|  9|            Caldeira|             Senegal|    Distribuição|
| 10|Nogueira Moura e ...|           Argentina|    Distribuição|
| 11|Castro Nascimento...|          Costa Rica|Pesquisa Clínica|
| 12|            Nogueira|                Fiji|Pesquisa Clínica|
| 13|    da Mota e Filhos

In [11]:
df_marketing = spark.read.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/campanhas_marketing.csv", header=True, inferSchema=True) 
df_marketing.createOrReplaceTempView("df_marketing")
df_marketing.show()

+---+-------------+-----------+----------+------------+--------------+
| id|nome_campanha|data_inicio|  data_fim|investimento|retorno_vendas|
+---+-------------+-----------+----------+------------+--------------+
|  1|   Campanha 1| 11-12-2024|13-10-2025|   203428.52|     129641.84|
|  2|   Campanha 2| 10-05-2025|13-10-2025|    90002.05|     204128.53|
|  3|   Campanha 3| 19-07-2025|13-10-2025|   371187.75|      96959.97|
|  4|   Campanha 4| 18-12-2024|13-10-2025|    200964.4|     555563.49|
|  5|   Campanha 5| 10-01-2025|13-10-2025|   195395.45|      21116.95|
|  6|   Campanha 6| 30-05-2025|13-10-2025|   138860.33|      43765.46|
|  7|   Campanha 7| 26-01-2025|13-10-2025|    216897.3|     551680.54|
|  8|   Campanha 8| 20-08-2025|13-10-2025|   127385.29|     340560.41|
|  9|   Campanha 9| 06-07-2025|13-10-2025|   384623.76|     423405.41|
| 10|  Campanha 10| 22-05-2025|13-10-2025|   455877.66|     269408.08|
| 11|  Campanha 11| 08-07-2025|13-10-2025|   405711.72|      76293.73|
| 12| 

In [12]:
df_prescricoes = spark.read.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/BRONZE/prescricoes.csv", header=True, inferSchema=True)
df_prescricoes.createOrReplaceTempView("df_prescricoes")
df_prescricoes.show()

+---+---------+-----------+----------+---------------+----------+
| id|medico_id|paciente_id|produto_id|data_prescricao|quantidade|
+---+---------+-----------+----------+---------------+----------+
|  1|       52|         30|        12|     27-09-2025|         3|
|  2|       55|         25|        37|     26-11-2024|         2|
|  3|       62|         57|        37|     17-12-2024|         2|
|  4|       50|          1|        43|     17-07-2025|         3|
|  5|        1|         88|        56|     03-02-2025|         1|
|  6|        5|         91|        10|     29-03-2025|         3|
|  7|       69|         75|        48|     30-11-2024|         1|
|  8|       26|        141|        47|     22-02-2025|         3|
|  9|       94|        134|        31|     25-06-2025|         3|
| 10|       48|         96|        23|     18-01-2025|         1|
| 11|        2|         28|         3|     07-10-2025|         1|
| 12|      115|         12|        15|     08-10-2025|         3|
| 13|     

resultado trimestral

In [13]:
df_resultado_trimestral = spark.sql("""
    SELECT 
        CASE
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (1, 2, 3) THEN 'Q1'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (4, 5, 6) THEN 'Q2'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (7, 8, 9) THEN 'Q3'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (10, 11, 12) THEN 'Q4'
        END AS trimestre,
        SUM(CAST(valor_total as decimal(10,2))) AS receita_total
    FROM df_vendas
    GROUP BY 
        CASE
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (1, 2, 3) THEN 'Q1'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (4, 5, 6) THEN 'Q2'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (7, 8, 9) THEN 'Q3'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) IN (10, 11, 12) THEN 'Q4'
        END
    ORDER BY trimestre ASC;
""")

df_resultado_trimestral.show()

df_resultado_trimestral.write.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/PRATA/resultado_trimestral.csv", header=True, mode="overwrite")



+---------+-------------+
|trimestre|receita_total|
+---------+-------------+
|       Q1|    812150.16|
|       Q2|    769777.00|
|       Q3|    804249.13|
|       Q4|    608030.44|
+---------+-------------+



Quantidade total de produtos vendidos por mês

In [14]:
df_resultado_mensal = spark.sql("""
    SELECT
        CASE 
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 1 THEN 'mes_1'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 2 THEN 'mes_2'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 3 THEN 'mes_3'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 4 THEN 'mes_4'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 5 THEN 'mes_5'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 6 THEN 'mes_6'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 7 THEN 'mes_7'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 8 THEN 'mes_8'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 9 THEN 'mes_9'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 10 THEN 'mes_10'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 11 THEN 'mes_11'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 12 THEN 'mes_12'
        END AS mes,
        SUM(CAST(valor_total AS DECIMAL(10,2))) AS receita_total
    FROM df_vendas
    GROUP BY 
        CASE 
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 1 THEN 'mes_1'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 2 THEN 'mes_2'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 3 THEN 'mes_3'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 4 THEN 'mes_4'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 5 THEN 'mes_5'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 6 THEN 'mes_6'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 7 THEN 'mes_7'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 8 THEN 'mes_8'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 9 THEN 'mes_9'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 10 THEN 'mes_10'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 11 THEN 'mes_11'
            WHEN MONTH(TO_DATE(data_venda, 'dd-MM-yyyy')) = 12 THEN 'mes_12'
        END
    ORDER BY mes ASC
""")

df_resultado_mensal.write.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/PRATA/resultado_mensal.csv", header=True, mode="overwrite")


Top 5 produtos mais vendidos

In [15]:
df_resultados_produtos = spark.sql("""
    SELECT
        p.id as id_produto,
        p.nome_produto,
        SUM(v.quantidade) AS total_vendido,
        CASE 
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 1 THEN 'mes_1'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 2 THEN 'mes_2'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 3 THEN 'mes_3'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 4 THEN 'mes_4'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 5 THEN 'mes_5'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 6 THEN 'mes_6'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 7 THEN 'mes_7'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 8 THEN 'mes_8'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 9 THEN 'mes_9'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 10 THEN 'mes_10'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 11 THEN 'mes_11'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 12 THEN 'mes_12'
        END AS mes
    FROM df_vendas v
    JOIN df_produtos p
        ON v.id = p.id
    GROUP BY
        p.id,
        p.nome_produto,
        CASE 
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 1 THEN 'mes_1'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 2 THEN 'mes_2'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 3 THEN 'mes_3'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 4 THEN 'mes_4'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 5 THEN 'mes_5'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 6 THEN 'mes_6'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 7 THEN 'mes_7'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 8 THEN 'mes_8'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 9 THEN 'mes_9'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 10 THEN 'mes_10'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 11 THEN 'mes_11'
            WHEN MONTH(TO_DATE(v.data_venda, 'dd-MM-yyyy')) = 12 THEN 'mes_12'
        END
    ORDER BY mes ASC;
    
""")
df_resultados_produtos.write.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/PRATA/resultado_mensal_produto.csv", header=True, mode="overwrite")


In [16]:
df_top5_produtos = spark.sql(""" 
    SELECT
        p.id as id_produto,
        p.nome_produto,
        Sum(v.quantidade) as total_vendido
        
    FROM df_produtos p
    JOIN df_vendas v 
        ON p.id = v.id
    GROUP BY 
        p.id, p.nome_produto
    ORDER BY 
        total_vendido DESC
      limit 5
          
          """)

In [17]:
df_top5_produtos.show()

+----------+------------+-------------+
|id_produto|nome_produto|total_vendido|
+----------+------------+-------------+
|        48|  Melatonina|           50|
|        36|     Clexane|           48|
|        23|Escitalopram|           47|
|        15|  Metformina|           46|
|        18|  Gliclazida|           45|
+----------+------------+-------------+



top 5 clientes com maior volume de compras

In [18]:
df_top_clientes = spark.sql("""
    SELECT 
        c.id AS id_cliente,
        c.nome_cliente,
        count(v.quantidade) AS total_compras
    FROM df_clientes c
    JOIN df_vendas v
        ON c.id = v.cliente_id
    GROUP BY c.id, c.nome_cliente
    ORDER BY total_compras DESC
""")
df_top_clientes.write.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/PRATA/top5_clientes.csv", header=True, mode="overwrite") 

Faturamento total por representante


In [19]:
spark.sql(""" 
               SELECT 
                     r.id AS id_representante,
                     r.nome_representante,
                     SUM(cast(v.valor_total AS DECIMAL(10,2))) AS receita_total
                FROM df_representantes r
                JOIN df_vendas v
                    ON r.id = v.representante_id
               GROUP BY r.id, r.nome_representante
               ORDER BY receita_total DESC
               LIMIT 5
           """).show()

+----------------+------------------+-------------+
|id_representante|nome_representante|receita_total|
+----------------+------------------+-------------+
|              29|      Sophia Porto|    125876.64|
|               6|    Eduardo Fogaça|    117905.33|
|              24|    Luana Monteiro|    111515.99|
|              39|João Vitor Azevedo|    106359.41|
|              33|       Breno Moura|    105250.00|
+----------------+------------------+-------------+



In [20]:
df_top5_representantes = spark.sql(""" 
               SELECT 
                     r.id AS id_representante,
                     r.nome_representante,
                     SUM(cast(v.valor_total AS DECIMAL(10,2))) AS receita_total
                FROM df_representantes r
                JOIN df_vendas v
                    ON r.id = v.representante_id
               GROUP BY r.id, r.nome_representante
               ORDER BY receita_total DESC
               LIMIT 5
           """)
df_top5_representantes.write.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/PRATA/top5_representantes.csv", header=True, mode="overwrite")

Percentual de atingimento da meta

In [21]:
df_metas_atingidas =spark.sql(""" 
                    SELECT
                        r.id AS id_representante,
                        r.nome_representante,
                        r.meta_mensal,
                        MONTH(TO_DATE(v.data_venda, 'dd-mm-yyyy')) AS mes,
                        SUM(CAST(v.valor_total AS DECIMAL(10,2))) AS receita_total,
                    CASE 
                        WHEN SUM(CAST(v.valor_total AS DECIMAL(10,2))) >= r.meta_mensal THEN 'Atingiu'
                            ELSE 'Não Atingiu'
                        END status_atingimento
                    FROM 
                        df_representantes r
                    JOIN
                        df_vendas v
                    ON 
                        r.id = v.representante_id
                    GROUP BY 
                        r.id, r.nome_representante, r.meta_mensal, MONTH(TO_DATE(v.data_venda, 'dd-mm-yyyy'))
""")
df_metas_atingidas.write.csv("/home/wells/Documentos/Ache Farmaceutica/Dados/PRATA/metas_atingidas.csv", header=True, mode="overwrite")

Melhores representantes

In [22]:
spark.sql(""" 
SELECT
    r.id AS id_representante,
    r.nome_representante,
    r.meta_mensal,
    MONTH(TO_DATE(v.data_venda, 'dd-mm-yyyy')) AS mes,
    SUM(CAST(v.valor_total AS DECIMAL(10,2))) AS receita_total,
CASE
    WHEN SUM(CAST(v.valor_total AS DECIMAL(10,2))) >= r.meta_mensal THEN 'Atingiu'
        ELSE 'Não Atingiu'
    END status_atingimento
FROM
    df_representantes r
JOIN
    df_vendas v
ON
    r.id = v.representante_id
GROUP BY
    r.id, r.nome_representante, r.meta_mensal, MONTH(TO_DATE(v.data_venda, 'dd-mm-yyyy'))
HAVING
    SUM(CAST(v.valor_total AS DECIMAL(10,2))) >= r.meta_mensal -- MUDANÇA AQUI: de WHERE para HAVING
ORDER BY
    receita_total DESC
""").show()

+----------------+------------------+-----------+---+-------------+------------------+
|id_representante|nome_representante|meta_mensal|mes|receita_total|status_atingimento|
+----------------+------------------+-----------+---+-------------+------------------+
|               6|    Eduardo Fogaça|   73662.02|  1|    117905.33|           Atingiu|
|              24|    Luana Monteiro|    80539.6|  1|    111515.99|           Atingiu|
|              33|       Breno Moura|   50531.85|  1|    105250.00|           Atingiu|
|              19|    Lorenzo Fogaça|    66055.4|  1|     87610.52|           Atingiu|
|              12|    Brenda Cardoso|   51173.47|  1|     78803.02|           Atingiu|
|              40|       Yago Barros|   51450.45|  1|     73338.48|           Atingiu|
+----------------+------------------+-----------+---+-------------+------------------+



Média de vendas mensais por produto


In [23]:
spark.sql("""
         SELECT
                p.id AS id_produto,
                p.nome_produto,
                AVG(v.quantidade) AS media_vendas_mensais
        FROM
                df_produtos p
        JOIN
                df_vendas v
            ON p.id = v.id  -- Corrected this based on common schema design
        GROUP BY
                p.id, p.nome_produto
         """).show()

+----------+-----------------+--------------------+
|id_produto|     nome_produto|media_vendas_mensais|
+----------+-----------------+--------------------+
|        49|       Cetirizina|                42.0|
|        31|            Engov|                43.0|
|        29|         Buscopan|                 4.0|
|         2|      Paracetamol|                39.0|
|         3|       Ibuprofeno|                12.0|
|        58|Buscopan Composto|                29.0|
|        40|       Nimesulida|                10.0|
|        53|       Cefalexina|                37.0|
|        22|       Fluoxetina|                 3.0|
|        34|          Eliquis|                24.0|
|        54|      Ceftriaxona|                 3.0|
|        59|       Ranitidina|                24.0|
|         7|        Omeprazol|                 7.0|
|        26|         Torsilax|                28.0|
|        30|           Dramin|                 5.0|
|        55|     Dexametasona|                39.0|
|        17|

Compare o faturamento de produtos antes e depois das campanhas.
Avalie se o investimento trouxe retorno (ROI positivo).

In [32]:

# testando novo script

spark.sql("""
    SELECT
        p.id AS id_produto,
        p.nome_produto,
        m.id AS id_campanha,
        m.nome_campanha,
        m.data_inicio,
        m.data_fim,
        m.investimento,  
                                                                                                 /* Calcula a receita antes e depois da campanha */                                                                                                 
    
    SUM(CASE 
        WHEN TO_DATE(v.data_venda, 'dd-MM-yyyy') < TO_DATE(m.data_inicio, 'dd-MM-yyyy') 
            THEN v.valor_total ELSE 0 END) AS receita_antes,
    
    SUM(CASE 
            WHEN TO_DATE(v.data_venda, 'dd-MM-yyyy') >= TO_DATE(m.data_inicio, 'dd-MM-yyyy') 
    
    AND
    
        TO_DATE(v.data_venda, 'dd-MM-yyyy') <= TO_DATE(m.data_fim, 'dd-MM-yyyy') 
            THEN v.valor_total ELSE 0 END) AS receita_depois,
    
                                                                                                          /* Calcula o ROI */
    
    CASE
        WHEN m.investimento > 0 
            THEN
                (SUM(CASE WHEN TO_DATE(v.data_venda, 'dd-MM-yyyy') >= TO_DATE(m.data_inicio, 'dd-MM-yyyy') 
    AND 
        TO_DATE(v.data_venda, 'dd-MM-yyyy') <= TO_DATE(m.data_fim, 'dd-MM-yyyy') 
            THEN v.valor_total ELSE 0 END) - SUM(CASE WHEN TO_DATE(v.data_venda, 'dd-MM-yyyy') < TO_DATE(m.data_inicio, 'dd-MM-yyyy') 
                THEN v.valor_total ELSE 0 END)) / m.investimento
            ELSE NULL
        END AS ROI
        
        
FROM 
        df_vendas v
JOIN
        df_produtos p ON v.id = p.id
JOIN
        df_marketing m ON p.id = m.id
    GROUP BY 
        p.id, p.nome_produto, m.id, m.nome_campanha, m.data_inicio, m.data_fim, m.investimento
    HAVING 
        ROI > 0
    ORDER BY ROI DESC
""").show()

+----------+-----------------+-----------+-------------+-----------+----------+------------+-------------+--------------+--------------------+
|id_produto|     nome_produto|id_campanha|nome_campanha|data_inicio|  data_fim|investimento|receita_antes|receita_depois|                 ROI|
+----------+-----------------+-----------+-------------+-----------+----------+------------+-------------+--------------+--------------------+
|        45|         Atenolol|         45|  Campanha 45| 10-12-2024|13-10-2025|    37458.79|          0.0|       8744.04| 0.23343092502454033|
|        59|       Ranitidina|         59|  Campanha 59| 21-10-2024|13-10-2025|    26420.04|          0.0|       2133.47|  0.0807519594974118|
|        35|        Apixabana|         35|  Campanha 35| 05-08-2025|13-10-2025|   251853.98|          0.0|       7502.57| 0.02978936445634093|
|        57|       Polaramine|         57|  Campanha 57| 17-01-2025|13-10-2025|   227050.86|          0.0|       5251.38|0.023128650558733848|


Quais especialidades médicas mais prescrevem medicamentos.


In [27]:
spark.sql(""" 
            SELECT 
                    m.especialidade,
                    COUNT(p.quantidade) AS total_prescricoes
          FROM 
            df_medicos m
          JOIN
            df_prescricoes p
              ON m.id = p.medico_id
          GROUP BY 
            m.especialidade
          ORDER BY total_prescricoes DESC
          
          """).show()

+-------------+-----------------+
|especialidade|total_prescricoes|
+-------------+-----------------+
|    Pediatria|               99|
|  Cardiologia|               80|
| Dermatologia|               68|
|    Ortopedia|               62|
|Clínico Geral|               57|
|  Ginecologia|               34|
+-------------+-----------------+



Médicos com maior número de prescrições.

In [17]:
spark.sql(""" 
            SELECT 
                    m.id as id_medico,
                    m.nome_medico,
                    m.especialidade,
                    COUNT(p.quantidade) AS total_prescricoes
          FROM 
            df_medicos m
          JOIN
            df_prescricoes p
              ON m.id = p.medico_id
          GROUP BY 
            m.id, m.nome_medico, m.especialidade
          ORDER BY total_prescricoes DESC
          
          """).show()

+---------+--------------------+-------------+-----------------+
|id_medico|         nome_medico|especialidade|total_prescricoes|
+---------+--------------------+-------------+-----------------+
|       94|     Emanuel Costela|  Cardiologia|                8|
|       85|Maria Vitória Mar...|    Pediatria|                7|
|       15|    Sra. Helena Lima|    Pediatria|                7|
|       80|           Ian Souza|    Pediatria|                7|
|        7|         Elisa Lopes| Dermatologia|                7|
|       89| Sra. Helena Cardoso|    Ortopedia|                6|
|       14|      Esther Cardoso|    Ortopedia|                6|
|       64|     Sra. Luiza Dias|    Pediatria|                6|
|        2|Dr. Davi Lucas Costa|Clínico Geral|                6|
|       52| Sra. Sarah Carvalho|  Cardiologia|                6|
|       59|  Larissa Cavalcanti|  Cardiologia|                6|
|       22|    Danilo Gonçalves|    Pediatria|                6|
|       84|      Leonardo


Quais produtos são mais prescritos por especialidade.


In [ ]:
spark.sql(""" 
            SELECT 
                pd.nome_produto,
                m.especialidade,
                COUNT(p.quantidade) AS total_prescricoes
            FROM
                df_prescricoes p
            JOIN
                df_medicos m ON p.medico_id = m.id
            JOIN
                df_produtos pd ON p.produto_id = pd.id
            GROUP BY
                pd.nome_produto,
                m.especialidade
                
            ORDER BY total_prescricoes DESC
          """).show()
# teste

+----------------+-------------+-----------------+
|    nome_produto|especialidade|total_prescricoes|
+----------------+-------------+-----------------+
|           Engov|  Cardiologia|                6|
|Insulina Regular|    Pediatria|                6|
|      Ibuprofeno|    Pediatria|                5|
|         Tylenol|    Ortopedia|                5|
|         Allegra| Dermatologia|                5|
|         Eliquis| Dermatologia|                4|
|      Furosemida|  Cardiologia|                4|
|         Eliquis|    Pediatria|                4|
|        Aspirina|    Pediatria|                4|
|        Rivotril|    Pediatria|                4|
|Insulina Regular|  Cardiologia|                4|
|      Metformina| Dermatologia|                4|
|      Simeticona| Dermatologia|                4|
|      Simeticona|    Ortopedia|                4|
|    Sinvastatina|    Pediatria|                3|
|     Cetoprofeno|    Pediatria|                3|
|      Furosemida|Clínico Geral